# Sea Surface

Concerns remote sensing products




## MODIS moderate resolution imaging spectrometer: Get data

* [NASA Earth Observations (NEO)](https://neo.sci.gsfc.nasa.gov/view.php?datasetId=MY1DMW_CHLORA&date=2019-02-01) site
  - No bounding box
  - Can select monthly averages or 8-day
  - Clicking on `Download raw data` produces 45MB files at 0.1 deg resolution as NetCDF
* MODIS Chlorophyll Level 1 and 2 data are [here](http://oceancolor.gsfc.nasa.gov/cgi/browse.pl?sen=am)
* MODIS Chlorophyll Level 3 data are [here](http://oceancolor.gsfc.nasa.gov/cgi/l3)


### mapping

How to emplace MODIS images on a map.


### archaea

- The image saver below does not work properly; I just get an empty image...
- The nbextension command should *not* be necessary but it is for the record:
  
```
jupyter nbextension enable --py --sys-prefix ipyleaflet
```

- install the MODIS WMS from [here](https://lpdaac.usgs.gov/data_access/web_map_services_wms)


In [6]:
import os, sys, time, glob, warnings
from IPython.display import clear_output             # use inside loop with clear_output(wait = True) followed by print(i)
warnings.filterwarnings('ignore')
this_dir = os.getcwd()

from matplotlib import pyplot as plt
from matplotlib import colors as mplcolors
import numpy as np, pandas as pd, xarray as xr
from numpy import datetime64 as dt64, timedelta64 as td64

print('\nJupyter Notebook running Python {}'.format(sys.version_info[0]))

# to do: explain what is going on here (?) archaic
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr’
# automatically reload external modules such as golive_library if they have changed
# %load_ext autoreload
# %autoreload 2


Jupyter Notebook running Python 3


In [3]:
ds_modis = xr.open_mfdataset()
ds_modis_0 = ds_modis[0]

print(ds_modis_0.MODISA_L3m_SST_2014_sst)
print(ds_modis_0.lat.values[0], modis.lon.values[0], modis.lat.values[-1], modis.lon.values[-1])
modisplot = ds_modis_0.MODISA_L3m_SST_2014_sst.plot(figsize=(12,12))

NameError: name 'g' is not defined

In [ ]:
# bug: This does not work properly yet

# plt.savefig(data + '/images/modis_SST.png')
import matplotlib
img=matplotlib.image.imread(image_d + 'modis_sst.png')
imgplot = plt.imshow(img)
# plt.show()

# The goal is to overlay an image (created earlier; .png file) with a controllable opacity
# 
# How I got to opacity control through a sequence of print statements: 
#   print(thisMap)
#   print(thisMap.layers)
#   print(thisMap.layers[1])                    which is cheating by hardcoding the layer index as 1
#   print(thisMap.layers[1].opacity)
#   therefore setting thisMap.layers[1].opacity = 0.2 works just fine

def ChangeMapOpacity(opacity):thisMap.layers[1].opacity = opacity

from ipyleaflet import Map, ImageOverlay, WMSLayer
opacity = 0.7
thisMap = Map(center=(47, -129), zoom=5, layout=Layout(width='100%', height='600px'))
# The path to the image for overlay is relative; I could not get an absolute path to work...
sourceImage =  '../data/images/modis_sst.png'
image_layer = ImageOverlay(url=sourceImage, bounds=((40.2, -138.58), (53.8, -122.45)), opacity=opacity)
thisMap.add_layer(image_layer)


interact(ChangeMapOpacity, opacity = widgets.FloatSlider(min=0., max=1., step=0.025, value=opacity, 
      continuous_update=False, display='Opacity'))

# It seems that 'thisMap' must be the last line of code in the cell, hence placing interact() above
thisMap

# The following works but does not track zoom with new higher resolution tiles
# wms = WMSLayer(url="https://demo.boundlessgeo.com/geoserver/ows?",layers="nasa:bluemarble")
# thisMap.add_layer(wms)
    

# This cell selects out MODIS-derived sea surface temperature in deg C at specific locations
#   This is for comparative work with in situ sensors in the Regional Cabled Observatory, which is
#   in turn part of the NSF-sponsored Ocean Observing Initiative

# OSB data coughs up 44.52897 -125.38966

eoLat, eoLon, eoDep          = 44. + 22./60. + 10./3600., -(124. + 57./60. + 15./3600.),  582.
osbLat, osbLon, osbDep       = 44. + 30./60. + 55./3600., -(125. + 23./60. + 23./3600.), 2906.
shrLat, shrLon, shrDep       = 44. + 34./60. +  9./3600., -(125  +  8./60. + 53./3600.),  778.
axbLat, axbLon, axbDep       = 45. + 49./60. +  5./3600., -(129. + 45./60. + 13./3600.), 2605.
ashesLat, ashesLon, ashesDep = 45. + 56./60. +  1./3600., -(130. +  0./60. + 50./3600.), 1543.
axcLat, axcLon, axcDep       = 45. + 57./60. + 17./3600., -(130. +  0./60. + 32./3600.), 1528.
axeLat, axeLon, axeDep       = 45. + 56./60. + 23./3600., -(129. + 58./60. + 27./3600.), 1516.
axiLat, axiLon, axiDep       = 45. + 53./60. + 35./3600., -(129. + 58./60. + 44./3600.), 1520.
ca = [('Endurance Offshore',                eoLat,    eoLon,    eoDep, 'Endurance Offshore'),
      ('Oregon Slope Base',                osbLat,   osbLon,   osbDep, 'Oregon Slope Base'),
      ('Southern Hydrate Ridge',           shrLat,   shrLon,   shrDep, 'Southern Hydrate Ridge'),
      ('Axial Base',                       axbLat,   axbLon,   axbDep, 'Axial Base'),
      ('Axial ASHES Vent Field',         ashesLat, ashesLon, ashesDep, 'Inferno vent'),
      ('Axial Caldera Center',             axcLat,   axcLon,   axcDep, 'Axial Caldera Center'),
      ('Axial Caldera East',               axeLat,   axeLon,   axeDep, 'Axial Caldera East'),
      ('Axial International Vent Field',   axiLat,   axiLon,   axiDep, 'Axial International Vent Field')
    ]

# for i in ca: print(i[0], i[1], i[2])
# for i in range(3): for j in range(3): print(m.MODISA_L3m_SST_2014_sst[0][i][j].values)
# 
# testLat = ca[1][1]       #   44.51528
# testLon = ca[1][2]       # -125.38972
# print(testLat, testLon)

sst = []
for i in range(len(ca)):
    # print(m.MODISA_L3m_SST_2014_sst.sel(lat=ca[i][1], lon=ca[i][2], method = 'nearest'))
    temp = modis.MODISA_L3m_SST_2014_sst.sel(lat=ca[i][1], lon=ca[i][2], method = 'nearest').values
    sst.append(round(float(temp[0]), 2))

# Here are MODIS-derived day-time sea surface temperatures at the cabled array locations on 2016-01-01T00:15:10 
print(sst)

# And this works properly albeit with limited tiling resolution
from ipyleaflet import Map, WMSLayer
wms = WMSLayer(
url="https://demo.boundlessgeo.com/geoserver/ows?",
layers="nasa:bluemarble"
)
m = Map(layers=(wms, ), center=(42.5531, -48.6914), zoom=3)
m

def pp(ds):
    ds['time'] = xr.Variable('time', [pd.Timestamp(ds.attrs['time_coverage_start']) + pd.Timedelta(15, unit='d')])
    return ds

cb=xr.open_mfdataset('modis_chlora_2017_*.nc', preprocess = pp, concat_dim='time').chlor_a.to_dataset()
# cb.chlor_a shows units as mg / cubic meter, numerically equivalent to ug / liter

p,a=plt.subplots(figsize=(13,9))

for x in np.arange(-125.38966-0.1, -125.38966+0.1, 0.1):
    for y in np.arange(44.52897-0.1, 44.52897+0.1, 0.1):
        a.plot(cb.time.values, cb.chlor_a.sel(lon=x, lat=y, method='nearest').values, 'o-')

# cb.to_netcdf('modis_chlora_5_month_timeseries.nc')

cb

latselect = np.arange(44.52897-1., 44.52897+1.00001, 0.10)
lonselect = np.arange(-125.38966-1., -125.38966+1.00001, 0.10)
local = cb.sel(lat = latselect, lon = lonselect, method = 'nearest')
local

%%time
p,a=plt.subplots(figsize=(13,9))
for i in range(9,12):
    for j in range(9,12):
        a.plot(local.time.values, local.chlor_a.isel(lat=i,lon=j).values, 'o-')

a.plot(local.time.values, local.chlor_a.isel(lat=10,lon=10).values, 'o-', linewidth=6)

local.to_netcdf('local.nc')

local=xr.open_dataset('local.nc')
local

l1

# p,a=plt.subplots(figsize=(13,9))
# for t in l1.chlor_a: print(t)  
# a.plot(local.time.values, t.values, 'o-')

for i in range(21): 
    for j in range(21): 
        print(l1.chlor_a[0:5, i, j].values)

local = cb.reindex_like()

local.chlor_a[0].plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=4.0)
plt.xlim(-128,-124); plt.ylim(43.5,45.5); plt.plot(-125.38966, 44.52897, '^')

local.chlor_a[1].plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=4.0)
plt.xlim(-128,-124); plt.ylim(43.5,45.5); plt.plot(-125.38966, 44.52897, '^')

local.chlor_a[2].plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=4.0)
plt.xlim(-128,-124); plt.ylim(43.5,45.5); plt.plot(-125.38966, 44.52897, '^')

local.chlor_a[3].plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=4.0)
plt.xlim(-128,-124); plt.ylim(43.5,45.5); plt.plot(-125.38966, 44.52897, '^')

local.chlor_a[4].plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=4.0)
plt.xlim(-128,-124); plt.ylim(43.5,45.5); plt.plot(-125.38966, 44.52897, '^')

ca=xr.open_rasterio('./chlora.tif').to_dataset(name='chlora')
ca



ca.chlora

cad=ca.where(ca.chlora<1.4)
cad.chlora.plot(figsize=(14,9),cmap=plt.cm.rainbow,vmin=0.00, vmax=1.0)
plt.xlim(-128,-124)
plt.ylim(43.5,45.5)
plt.plot(-125.38966, 44.52897, '^')
# a=p.axes()
# a.set(xlim=(-155,-135))

float(ca.sel(band=0,y=44.52897,x=-125.38966, method='nearest').values)

for x in np.arange(-125.5, -125.0, 0.1):
    for y in np.arange(44.3, 44.8, 0.1):
        print(float(ca.sel(band=0,y=y,x=x, method='nearest').values))